In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep

options= Options()
options.headless = True
driver = webdriver.Chrome(options = options)
url= "https://www.customs.go.jp/news/index.htm"

driver.get(url)
sleep(1)

soup = BeautifulSoup(driver.page_source, 'lxml')
soup = soup.find("div", attrs={"class":"pastList category"}).find("ul").find("li").find("h3")

In [70]:
#가장 최근 뉴스가 올라온 날짜 가져오기
dates=[]
date = soup.find("span").get_text()
dates.append(date)
print(dates[0])

7月26日


In [74]:
#가장 최근 뉴스 태그들 가져오기
tags=[]
tag = soup.select("img")
for i in range(len(tag)):
    if len(tag[i]['alt']) > 0:
        tags.append(tag[i]['alt'])
print(tags[0])

['その他', '手続']


In [73]:
#가장 최근 뉴스의 제목과 링크 가져오기
topics=[]
links=[]
data = soup.find("a")

topic = data.get_text()
link = "https://www.customs.go.jp/english"+data["href"][2:]

topics.append(topic)
links.append(link)

print(topics)
print(links)

['税関における新型コロナウイルス感染症陽性者の判明について(令和4年7月26日更新)']
['https://www.customs.go.jp/englishnews/2022covid-19.html']


In [75]:
#가장 최근 뉴스 날짜, 태그, 제목, 링크 DF형태로 출력
import pandas as pd

df = pd.DataFrame()
df['date'] = dates
df['tag'] = tags[0]
df['topic'] = topics
df['link'] = links
df

,date,tag,topic,link
0,7月26日,その他,税関における新型コロナウイルス感染症陽性者の判明について(令和4年7月26日更新),https://www.customs.go.jp/englishnews/2022covi...


In [94]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep

options= Options()
options.headless = True
driver = webdriver.Chrome(options = options)
url= "https://www.customs.go.jp/news/index.htm"

driver.get(url)
sleep(1)

soup = BeautifulSoup(driver.page_source, 'lxml')
soup = soup.find("div", attrs={"class":"pastList category"})

In [96]:
#모든 date list에 추가
dates = []
rows = soup.find_all("h3")
for row in rows: #h3 태그 하나씩 확인
    A = row.find_all("a") #h3 태그 하나에 여러 값이 있는 경우를 위해 a 태그 찾기
    for a in A:
        date = row.find("span").get_text()
        dates.append(date)
print(len(dates))

208


In [97]:
#모든 tag list에 추가
tags = []
tag = soup.find_all("img")
t=['統計', '手続', '採用', '法令', 'その他', '報道資料']
#   통계,  수속(절차), 채용,  법령, 기타,  보도자료

for i in range(len(tag)):
    if tag[i]['alt'] in t:
        tags.append(tag[i]['alt'])
print(len(tags))

208


In [98]:
#모든 topic, link list에 추가
topics = []
links = []

datas = soup.find_all("a")

for data in datas:
    topics.append(data.get_text())
    links.append("https://www.customs.go.jp/english"+data["href"][2:])
print(len(topics))
print(len(links))

208
208


In [99]:
import pandas as pd

df = pd.DataFrame()
df['date'] = dates
df['tag'] = tags
df['topic'] = topics
df['link'] = links
df

,date,tag,topic,link
0,7月26日,その他,税関における新型コロナウイルス感染症陽性者の判明について(令和4年7月26日更新),https://www.customs.go.jp/englishnews/2022covi...
1,7月26日,手続,価格の換算に用いる外国為替相場（令和4年7月31日～令和4年8月6日）,https://www.customs.go.jp/english/tetsuzuki/ka...
2,7月21日,その他,クレジットカードを利用した関税等の納付のページを更新しました。,https://www.customs.go.jp/english/kaigairyoko/...
3,7月21日,その他,スマートフォン決済アプリを利用した関税等の納付のページを更新しました。,https://www.customs.go.jp/english/kaigairyoko/...
4,7月21日,統計,貿易統計（令和4年６月分速報）,https://www.customs.go.jp/englishoukei/latest/...
...,...,...,...,...
203,1月6日,その他,沖縄地区税関における新型コロナウイルス感染症陽性者の判明について,https://www.customs.go.jp/englishnews/covid-19...
204,1月6日,その他,沖縄地区税関における新型コロナウイルス感染症陽性者の判明について,https://www.customs.go.jp/englishnews/covid-19...
205,1月4日,その他,沖縄地区税関における新型コロナウイルス感染症陽性者の判明について,https://www.customs.go.jp/englishnews/covid-19...
206,1月1日,手続,実行関税率表（2022年1月1日版）,https://www.customs.go.jp/english/tariff/2022_...


In [100]:
df.to_csv('jc_new.csv', mode='w', encoding='utf-8-sig')